In [ ]:
and #@title EligibilityTrace

import numpy as np
import numpy as np
import random

def choose_action(state, q_table, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, q_table.shape[2] - 1)
    else:
        return np.argmax(q_table[int(state[0]), int(state[1])])

def initialize_q_table(grid_size, num_actions):
    return np.zeros(grid_size + (num_actions,))

def initialize_eligibility_table(grid_size, num_actions):
    return np.zeros(grid_size + (num_actions,))

def update_q_table(q_table, eligibility_trace, current_state, action, reward, next_state, next_action, alpha, gamma, lambd):
    current_q = q_table[current_state + (action,)]
    next_q = q_table[next_state + (next_action,)]
    td_error = reward + gamma * next_q - current_q

    # Update eligibility trace
    eligibility_trace[current_state + (action,)] += 1

    # # Update Q-values and decay eligibility trace
    # for i in range(q_table.shape[0]):
    #     for j in range(q_table.shape[1]):
    #         for a in range(q_table.shape[2]):
    #             q_table[i, j, a] += alpha * td_error * eligibility_trace[i, j, a]
    #             eligibility_trace[i, j, a] *= gamma * lambd
    # Update Q-values and decay eligibility trace
    q_table += alpha * td_error * eligibility_trace
    eligibility_trace *= gamma * lambd

def derive_policy(q_table):
    policy = np.argmax(q_table, axis=2)
    return policy

def eligibility_trace_method(grid_size, num_actions, alpha, gamma, lambd, transitions):
    q_table = initialize_q_table(grid_size, num_actions)
    eligibility_trace = initialize_eligibility_table(grid_size, num_actions)

    for (i, j, action, reward, m, n) in transitions:
        i, j, action, reward, m, n = int(i), int(j), int(action), int(reward), int(m), int(n)
        current_state = (i, j)
        next_state = (m, n)
        next_action = choose_action(next_state, q_table, epsilon)

        update_q_table(q_table, eligibility_trace, current_state, action, reward, next_state, next_action, alpha, gamma, lambd)

    return derive_policy(q_table)

# Parameters
grid_size = (n_x, n_y)  # Example grid size
num_actions = 4     # 4 actions: up, down, left, right
alpha = 0.1         # Learning rate
gamma = 0.99        # Discount factor
lambd = 0.8         # Eligibility trace decay factor
epsilon = 0.1  # Exploration rate

# Train the agent with the data
import pickle
with open('sorted_samples.pkl', 'rb') as file:
    data = pickle.load(file)
data = [tuple(row) for row in data]

# Run SARSA(λ)
policy = eligibility_trace_method(grid_size, num_actions, alpha, gamma, lambd, data)
